### Импорт библиотек, настройка 



In [1]:
import torch
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

c:\Users\minik\anaconda3\envs\PyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.2.2
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce GTX 1650 Ti


### Create DataSet, Preproccesing, Augmentation, DataLoader


In [4]:
# Пути данных
TRAIN_DATA = "C:\\Users\\minik\\Desktop\\datasets\\1\\"



In [5]:
#Задаем параменты аугментации
transform = A.Compose([
    A.OneOf([A.RandomRotate90(), A.Transpose()], p=0.5),
    A.OneOf([A.HorizontalFlip(p=1), A.VerticalFlip(p=1)], p=0.5),
    A.OneOf([A.RandomBrightnessContrast(p=0.5), A.RandomGamma(p=0.8) ], p=0.5),
    A.OneOf([A.MedianBlur(blur_limit=3, p=0.1), A.GaussNoise()], p=0.2),
    A.OneOf([A.GridDistortion(p=0.5)], p=0.3),
    A.CLAHE(p=0.8),
    A.RandomBrightnessContrast(p=0.8),
    A.RandomGamma(p=0.8)
])


In [7]:
#Создаем класс для создания датасета, в котором используем аугментацию
#данных с помощью библиотеки Albumentation

class CustomDataset_AUG(Dataset):
    def __init__(self, img_path, msk_path, transform=None):
        self.images_paths = os.path.join(img_path, 'images')
        self.masks_paths = os.path.join(msk_path, 'masks\\Tubulitis')
        self.image_filenames = os.listdir(self.images_paths)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.images_paths, image_name)
        mask_path = os.path.join(self.masks_paths, image_name)


        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("RGB")

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image = augmented['image']
            mask = augmented['mask']


        image = transforms.ToTensor()(image)
        mask = torch.from_numpy(np.array(mask, dtype=np.float32) / 255.0)

        return image, mask

In [ ]:
# Создаем Augmented_DS
Augmented_DS = CustomDataset_AUG(
    img_path = TRAIN_DATA,
    msk_path = TRAIN_DATA,
    transform = transform
)

DIR_TO_AUG_IMAGE = "C:\\Users\\minik\\Desktop\\datasets\\1\\aug\\images"  # путь для сохранения аугментированного изображения
DIR_TO_AUG_MASK =  "C:\\Users\\minik\\Desktop\\datasets\\1\\aug\\masks\\Tubulitis"  # путь для сохранения аугментированной маски

num_augmentations = 2

for i in range(num_augmentations):

  for idx, (image, mask) in enumerate(Augmented_DS):
      # Convert tensor to numpy array
      image_np = image.permute(1, 2, 0).numpy()
      mask_np = mask.numpy()

      # Save augmented image and mask
      cv2.imwrite(
          os.path.join(DIR_TO_AUG_IMAGE, f"{idx}_{i}.png"),
          cv2.cvtColor((image_np * 255).astype(np.uint8),
          cv2.COLOR_RGB2BGR))
      cv2.imwrite(
          os.path.join(DIR_TO_AUG_MASK, f"{idx}_{i}.png"),
          (mask_np * 255).astype(np.uint8))